In [49]:
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.optimize as opt
import ot  # Для транспортной метрики
from scipy.stats import uniform, truncnorm
from scipy.optimize import differential_evolution


In [50]:
#pip install pot
!pip install  PyQt5 


In [3]:
#!pip install PySide2

ERROR: Could not find a version that satisfies the requirement PySide2 (from versions: none)
ERROR: No matching distribution found for PySide2


In [51]:
%matplotlib qt

In [52]:
def plot_3d(C):
    # Create figure and 3D axis
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    
    ax.set_zlim(22, 0)

    # Plot the voxels where value == 1
    ax.voxels(C, edgecolor='k')

In [53]:
def compare_XY(X, Y):
    # Create figure and 3D axis
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    
    ax.set_zlim(22, 0)

    # Plot the voxels where value == 1
    ax.voxels(X, edgecolor='k')
    ax.voxels(Y, edgecolor='r')

In [54]:
def compare_proj(X, Y):
    # Create figure and 3D axis
    fig, ax = plt.subplots(1, 2)
    
    ax[0].matshow(X.any(axis=0).transpose() + 5 * Y.any(axis=0).transpose() , cmap='Greys')
    ax[1].matshow(X.any(axis=1).transpose() + 5 * Y.any(axis=1).transpose() , cmap='Greys')
    
    ax[0].set_aspect(96 / 22)
    ax[0].set_xlim(0, 96)
    ax[0].set_ylim(22, 0)
    ax[1].set_aspect(96 / 22)
    ax[1].set_xlim(0, 96)
    ax[1].set_ylim(22, 0)

In [55]:
def x_proj(C):
    return C.any(axis=0)

def y_proj(C):
    return C.any(axis=1)

In [56]:
def plot_X_projection(C):
    plt.matshow(C.any(axis=0).transpose(), cmap='Greys')

def plot_Y_projection(C):
    plt.matshow(C.any(axis=1).transpose(), cmap='Greys')
    
def plot_projections(C):
    fig, ax = plt.subplots(1, 2)
    
    ax[0].matshow(C.any(axis=0).transpose(), cmap='Greys')
    ax[1].matshow(C.any(axis=1).transpose(), cmap='Greys')
    
    ax[0].set_aspect(96 / 22)
    ax[0].set_xlim(0, 96)
    ax[0].set_ylim(22, 0)
    ax[1].set_aspect(96 / 22)
    ax[1].set_xlim(0, 96)
    ax[1].set_ylim(22, 0)

In [57]:
def check_XY_bounds(x, xmin=0, xmax=95):
    return (x >= xmin) & (x <= xmax)

### Опишем модель взаимодействия

In [58]:
# # Опишем модель "звезды", т.е. у нас есть набор параметров:
# # точка и углы влёта, точка взаимодействия и направления разлёта частиц.
# # Мы подгоняем параметры этой модели под данные.

# startx, starty, startz = 34.0, 45.0, 0.0
# theta = np.pi / 4
# phi = np.pi / 3

# line = (startx, starty, startz, theta, phi)  # Набор, определяющий прямую
# l = (np.tan(theta) * np.cos(phi), np.tan(theta) * np.sin(phi), 1.0)  # Направляющий вектор прямой с координатой z = 1

In [59]:
# # Это у нас одна прямая
# lx = startx + l[0] * np.arange(0, 22, 1)
# ly = starty + l[1] * np.arange(0, 22, 1)
# lz = np.arange(0, 22, 1)

In [60]:
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')

# ax.plot(lx, ly, lz, '-ok')
# ax.set_zlim(np.max(lz), np.min(lz))

In [61]:
# # Теперь надо описать модель звезды

# zint = 7  # Точка взаимодействия
# npart = 3   # Количество порожденных частиц (возможно, эту величину стоит сделать стохастической)
# maxz = 22  # Глубина калориметра
# thetapart = np.array([0, 2 * np.pi / 3, np.pi / 3])
# phipart = np.array([0.0, np.pi / 6, -np.pi / 4])
# direction = thetapart < np.pi / 2  # Вверх или вниз летит частица

In [62]:
# line = (startx, starty, startz, theta, phi)  # Набор, определяющий прямую
# l = (np.tan(theta) * np.cos(phi), np.tan(theta) * np.sin(phi), 1.0)  # Направляющий вектор прямой с координатой z = 1

# # Это у нас одна прямая
# lx = startx + l[0] * np.arange(0, zint + 1, 1)
# ly = starty + l[1] * np.arange(0, zint + 1, 1)
# lz = np.arange(0, zint + 1, 1)

In [63]:
# # Координаты X, Y точки взаимодействия
# xint, yint = lx[zint], ly[zint]

In [64]:
# lines = dict()

# for line_num in range(npart):
#     if direction[line_num]:
#         steps = maxz - zint + 1
#         lines[line_num] = [
#             xint + np.tan(thetapart[line_num]) * np.cos(phipart[line_num]) * np.arange(0, steps - 1, 1),
#             yint + np.tan(thetapart[line_num]) * np.cos(phipart[line_num]) * np.arange(0, steps - 1, 1),
#             np.arange(zint, maxz, 1)
#         ]
#     else:
#         steps = zint + 1
#         lines[line_num] = [
#             xint - np.tan(thetapart[line_num]) * np.cos(phipart[line_num]) * np.arange(0, steps, 1),
#             yint - np.tan(thetapart[line_num]) * np.cos(phipart[line_num]) * np.arange(0, steps, 1),
#             np.arange(zint, -1, -1)
#         ]

In [65]:
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')

# ax.plot(lx, ly, lz, '-ok')

# for line_num in range(npart):
#     ax.plot(*lines[line_num], '-or')
    
# ax.plot([xint], [yint], [zint], 'ok', ms=20)


# ax.set_zlim(22, 0)
# ax.set_xlim(-11, 11)
# ax.set_ylim(-11, 11)

### Процедуры генерации событий

In [66]:
# StartX 0..95, StartY 0..95, start_theta 0..?, start_phi -pi..pi
def _generate_event(startx, starty, start_theta, start_phi, zint, npart, theta_part, phi_part):
    startz = 0
    maxz = 21
    l = (np.tan(start_theta) * np.cos(start_phi), np.tan(start_theta) * np.sin(start_phi), 1)  # Направляющий вектор прямой с координатой z = 1
    lx = startx + l[0] * np.arange(0, zint + 1, 1)
    ly = starty + l[1] * np.arange(0, zint + 1, 1)
    lz = np.arange(0, zint + 1, 1, dtype=int)
    zint = round(zint)
    xint, yint = lx[zint], ly[zint]  # Координаты X, Y точки взаимодействия
    
    C = np.zeros((96, 96, 22), dtype=int)
    
    # Флаг прерванного трека (вылет за пределы калориметра) 
    track_interrupted = False
    
    # Проверка того, вышли ли мы за пределы калориметра
    # Если вылетели, то обрываем траекторию
    if not(check_XY_bounds(xint) and check_XY_bounds(yint)): 
        idx = check_XY_bounds(lx) & check_XY_bounds(ly)
        lx, ly, lz = lx[idx], ly[idx], lz[idx]
        track_interrupted = True
        
#     print(lx, ly)
#     print(xint, yint, zint)
    
    lx_int = np.round(lx).astype(int)
    ly_int = np.round(ly).astype(int)
    
    C[lx_int, ly_int, lz] = 1  # Добавили траекторию первичной частицы
    
    
    if not(track_interrupted):
        lines = dict()
        
        direction = np.array(theta_part) < np.pi / 2

        for line_num in range(npart):
            newline = []
            if direction[line_num]:
                steps = maxz - zint + 1
                newline = [
                    xint + np.tan(theta_part[line_num]) * np.cos(phi_part[line_num]) * np.arange(0, steps - 1, 1),
                    yint + np.tan(theta_part[line_num]) * np.cos(phi_part[line_num]) * np.arange(0, steps - 1, 1),
                    np.arange(zint, maxz, 1, dtype=int)
                ]
            else:
                steps = zint + 1
                newline = [
                    xint - np.tan(theta_part[line_num]) * np.cos(phi_part[line_num]) * np.arange(0, steps, 1),
                    yint - np.tan(theta_part[line_num]) * np.cos(phi_part[line_num]) * np.arange(0, steps, 1),
                    np.arange(zint, -1, -1, dtype=int)
                ]
            # Отрезаем значения вне прибора
            idx = (newline[0] >= 0) & (newline[0] <= 95) & (newline[1] >= 0) & (newline[1] <= 95)
            lines[line_num] = [newline[0][idx], newline[1][idx], newline[2][idx]]



        for line_num in range(npart):
            C[np.round(lines[line_num][0]).astype(int),
              np.round(lines[line_num][1]).astype(int),
              lines[line_num][2]] = 1
    
    return C

In [67]:
def _generate_zero_event(startx, starty, theta, phi):
    return _generate_event(startx, starty, theta, phi, 21, 0, [], [])

def _generate_one_event(startx, starty, theta, phi, zint, theta1, phi1):
    return _generate_event(startx, starty, theta, phi, zint, 1, [theta1], [phi1])

def _generate_two_event(startx, starty, theta, phi, zint, theta1, phi1, theta2, phi2):
    return _generate_event(startx, starty, theta, phi, zint, 2, [theta1, theta2], [phi1, phi2])

def _generate_three_event(startx, starty, theta, phi, zint, theta1, phi1, theta2, phi2, theta3, phi3):
    return _generate_event(startx, starty, theta, phi, zint, 3, [theta1, theta2, theta3], [phi1, phi2, phi3])

def _generate_four_event(startx, starty, theta, phi, zint, theta1, phi1, theta2, phi2, theta3, phi3, theta4, phi4):
    return _generate_event(
        startx, starty, theta, phi, zint,
        4,
        [theta1, theta2, theta3, theta4],
        [phi1, phi2, phi3, phi4]
    )


In [68]:
def plot_3d_event(startx, starty, start_theta, start_phi, zint, npart, theta_part, phi_part):
    X = _generate_event(startx, starty, start_theta, start_phi, zint, npart, theta_part, phi_part)
    plot_3d(X)

In [69]:
X = _generate_event(34.0, 45.0, np.pi / 4, np.pi / 3, 7, 3, [0, 2 * np.pi / 3, np.pi / 3], [0.0, np.pi / 6, -np.pi / 4])
Y = _generate_event(34.0, 52.0, 0, 0, 21, 0, [], [])
Z = _generate_event(52.0, 34.0, 0, 0, 21, 0, [], [])
A = _generate_event(52.0, 34.0, 0, 0, 6, 1, [np.pi/6], [np.pi/6])
B = _generate_event(46.0, 46.0, np.pi / 2 * 0.9, 0, 21, 0, [], []) 

In [70]:
# Сделаем выборку тестовых событий
size = 100

test_0_params = dict()
test_0_events = dict()

test_1_params = dict()
test_1_events = dict()

test_2_params = dict()
test_2_events = dict()

test_3_params = dict()
test_3_events = dict()

# Событий с точкой взаимодействия в середине должно быть больше
def generate_startx(size=100):
    loc, scale = 47.5, 20.0
    # Define bounds (in standard normal space)
    lower, upper = -loc / scale, loc / scale
    samples = truncnorm.rvs(lower, upper, loc=loc, scale=scale, size=size)
    integers = np.round(samples)
    return integers

# Событий с точкой взаимодействия в середине должно быть больше
def generate_zint(size=100):
    # Define bounds (in standard normal space)
    lower, upper = (0 - 10.5) / 4.0, (21 - 10.5) / 4.0
    samples = truncnorm.rvs(lower, upper, loc=10.5, scale=4.0, size=size)
    integers = np.round(samples).astype(int)
    return integers

# Азимутальный угол распределён равномерно
def generate_phi_angle(size=100):
    samples = uniform.rvs(0, np.pi * 2, size=size)
    return samples

# Зенитный угол распределён равномерно
def generate_theta_start_angle(size=100):
    scale = 0.3
    # Define bounds (in standard normal space)
    lower, upper = -np.pi / 3 / scale, np.pi / 3 / scale
    samples = truncnorm.rvs(lower, upper, loc=0, scale=scale, size=size)
    return np.abs(samples)

# Зенитный угол распределён равномерно
def generate_theta_int_angle(size=100):
    samples = uniform.rvs(-np.pi / 2, np.pi / 2, size=size)
    return samples


# Zero events (no interaction)
startx = generate_startx(size=size)
starty = generate_startx(size=size)
theta = generate_theta_start_angle(size=size)
phi = generate_phi_angle(size=size)

test_0_params = [(startx[i], starty[i], theta[i], phi[i]) for i in range(size)]
test_0_events = [_generate_zero_event(*test_0_params[i]) for i in range(size)] 
    
# One particle events
startx = generate_startx(size=size)
starty = generate_startx(size=size)
theta = generate_theta_start_angle(size=size)
phi = generate_phi_angle(size=size)
zint = generate_zint(size=size)
theta1 = generate_theta_int_angle(size=size)
phi1 = generate_phi_angle(size=size)

test_1_params = [(startx[i], starty[i], theta[i], phi[i], zint[i], theta1[i], phi1[i]) for i in range(size)]
test_1_events = [_generate_one_event(*test_1_params[i]) for i in range(size)] 

# Two particle events
startx = generate_startx(size=size)
starty = generate_startx(size=size)
theta = generate_theta_start_angle(size=size)
phi = generate_phi_angle(size=size)
zint = generate_zint(size=size)
theta1 = generate_theta_int_angle(size=size)
phi1 = generate_phi_angle(size=size)
theta2 = generate_theta_int_angle(size=size)
phi2 = generate_phi_angle(size=size)

test_2_params = [(startx[i], starty[i], theta[i], phi[i], zint[i], theta1[i], phi1[i], theta2[i], phi2[i]) for i in range(size)]
test_2_events = [_generate_two_event(*test_2_params[i]) for i in range(size)] 

# Three particle events
startx = generate_startx(size=size)
starty = generate_startx(size=size)
theta = generate_theta_start_angle(size=size)
phi = generate_phi_angle(size=size)
zint = generate_zint(size=size)
theta1 = generate_theta_int_angle(size=size)
phi1 = generate_phi_angle(size=size)
theta2 = generate_theta_int_angle(size=size)
phi2 = generate_phi_angle(size=size)
theta3 = generate_theta_int_angle(size=size)
phi3 = generate_phi_angle(size=size)

test_3_params = [(startx[i], starty[i], theta[i], phi[i], zint[i], theta1[i], phi1[i], theta2[i], phi2[i], theta3[i], phi3[i]) for i in range(size)]
test_3_events = [_generate_three_event(*test_3_params[i]) for i in range(size)] 

In [71]:
plot_3d(test_3_events[55])

In [15]:
# запускать по желанию 
for i in range(0, 100, 10):
    plot_3d(test_3_events[i])

In [16]:
for i in range(0, 100, 10):
    plot_3d(test_2_events[i])

/var/folders/qw/st67mykd16sby063_l_fglkr0000gn/T/ipykernel_31228/3213367862.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize=(10, 8))


In [72]:
plot_3d(A)

In [37]:
plot_3d(B)

In [18]:
plot_3d(X)

In [19]:
plot_3d(Y)

## Теперь нужно написать целевую функцию: близость модели к исходным данным

In [73]:
# Будем минимизировать транспортную метрику (Вассерштайна)
# Она энергозатратная, но для разреженных матриц вроде бы не критично
def wasserstein_distance(mat1, mat2):
    # Get coordinates of 1s in each matrix
    coords1 = np.argwhere(mat1 == 1)  # shape (N, 3)
    coords2 = np.argwhere(mat2 == 1)  # shape (M, 3)
    
    if len(coords1) == 0 or len(coords2) == 0:
        distance = np.inf
    else:
        # Compute Euclidean cost matrix (distance between all pairs)
        cost_matrix = ot.dist(coords1, coords2, metric='euclidean')

        # Uniform weights (since all 1s are equally important)
        weights1 = np.ones(len(coords1)) / len(coords1)
        weights2 = np.ones(len(coords2)) / len(coords2)

        # Compute Wasserstein distance
        distance = ot.emd2(weights1, weights2, cost_matrix)
    return distance

In [74]:
# Расстояние Хэмминга (симметрическая разность)
def hamming_distance(mat1, mat2):
    return np.sum(mat1 != mat2)

In [75]:
# from skimage.metrics import structural_similarity as ssim

# # Структурная схожесть (Structural Similarity) -- вообще никак не работает
# def ssim(mat1, mat2):
#     return np.sum(mat1 != mat2)

In [76]:
# Функция для сравнения с линией
def _objective_zero(params, to_x, to_y):
    startx, starty, theta, phi = params
    E = _generate_event(startx, starty, theta, phi, 21, 0, [], [])
    return wasserstein_distance(to_x, x_proj(E)) + wasserstein_distance(to_y, y_proj(E))
#     return ssim(EX, to_x, data_range=1.0) + ssim(EY, to_y, data_range=1.0)

In [77]:
# Функция для сравнения со случаем, когда порождена одна частица
def _objective_one(params, to_x, to_y):
    startx, starty, theta, phi, zint, theta1, phi1 = params
    E = _generate_event(startx, starty, theta, phi, zint, 1, [theta1], [phi1])
    return wasserstein_distance(to_x, x_proj(E)) + wasserstein_distance(to_y, y_proj(E))

# def plot_3d_result_one(params):
#     startx, starty, theta, phi, zint, theta1, phi1 = params
#     X = _generate_event(startx, starty, theta, phi, zint, 1, [theta1], [phi1])
#     plot_3d(X)

In [78]:
# Функция для сравнения со случаем, когда порождены две частицы
def _objective_two(params, to_x, to_y):
    startx, starty, theta, phi, zint, theta1, phi1, theta2, phi2 = params
    E = _generate_event(startx, starty, theta, phi, zint, 2, [theta1, theta2], [phi1, phi2])
    return wasserstein_distance(to_x, x_proj(E)) + wasserstein_distance(to_y, y_proj(E))

# def plot_3d_result_two(params):
#     startx, starty, theta, phi, zint, theta1, phi1, theta2, phi2 = params
#     X = _generate_event(startx, starty, theta, phi, zint, 2, [theta1, theta2], [phi1, phi2])
#     plot_3d(X)

In [79]:
# Функция для сравнения со случаем, когда порождены три частицы
def _objective_three(params, to_x, to_y):
    startx, starty, theta, phi, zint, theta1, phi1, theta2, phi2, theta3, phi3 = params
    E = _generate_event(startx, starty, theta, phi, zint, 3, [theta1, theta2, theta3], [phi1, phi2, phi3])
    return wasserstein_distance(to_x, x_proj(E)) + wasserstein_distance(to_y, y_proj(E))

#     return hamming_distance(to_x, EX) + hamming_distance(to_y, EY)
#     return -ssim(EX, to_x, data_range=1.0) - ssim(EY, to_y, data_range=1.0)

# def plot_3d_result_three(params):
#     startx, starty, theta, phi, zint, theta1, phi1, theta2, phi2, theta3, phi3 = params
#     X = _generate_event(startx, starty, theta, phi, zint, 2, [theta1, theta2, theta3], [phi1, phi2, phi3])
#     plot_3d(X)

In [80]:
# Функция для сравнения со случаем, когда порождены три частицы
def _objective_four(params, target_mask):
    if len(params) != 13:
        return 1e6
    try:
        gen_mask = _generate_four_event(*params) > 0
        return wasserstein_distance(gen_mask, target_mask)
    except Exception as e:
        print("Ошибка:", e)
        return 1e6

In [81]:
import numpy as np
from typing import Sequence

# ---------------------------------------------------------------------
# 0. Базовая универсальная функция (она уже у нас есть, оставляю без изменений)
# ---------------------------------------------------------------------
def _generate_event(startx: float, starty: float,
                    theta0: float, phi0: float, zint: float,
                    npart: int,
                    theta_part: Sequence[float],
                    phi_part: Sequence[float]) -> np.ndarray:
    """
    Генерирует бинарную маску (96, 96, 44) события:
    - primary идёт до слоя int(zint)-1
    - каждая secondary идёт до z=43
    """
    mask = np.zeros((96, 96, 44), dtype=np.uint8)

    def propagate(x0, y0, theta, phi, z_stop):
        z, x, y = 0, x0, y0
        while (
            z < z_stop
            and 0 <= z < 44
            and 0 <= x < 96
            and 0 <= y < 96
        ):
            mask[int(x), int(y), int(z)] = 1
            x += np.tan(theta) * np.cos(phi)
            y += np.tan(theta) * np.sin(phi)
            z += 1

    propagate(startx, starty, theta0, phi0, max(int(zint) - 1, 0))

    for th, ph in zip(theta_part, phi_part):
        propagate(startx, starty, th, ph, 44)

    return mask  # (96, 96, 44)


# ---------------------------------------------------------------------
# 1. Узкие «обёртки» под 0–4 вторичных частиц
# ---------------------------------------------------------------------
def _generate_zero_event(startx, starty, theta0, phi0, zint):
    return _generate_event(startx, starty, theta0, phi0, zint, 0, [], [])

def _generate_one_event(startx, starty, theta0, phi0, zint,
                        theta1, phi1):
    return _generate_event(startx, starty, theta0, phi0, zint,
                           1, [theta1], [phi1])

def _generate_two_event(startx, starty, theta0, phi0, zint,
                        theta1, phi1, theta2, phi2):
    return _generate_event(startx, starty, theta0, phi0, zint,
                           2, [theta1, theta2], [phi1, phi2])

def _generate_three_event(startx, starty, theta0, phi0, zint,
                          theta1, phi1, theta2, phi2, theta3, phi3):
    return _generate_event(startx, starty, theta0, phi0, zint,
                           3, [theta1, theta2, theta3],
                           [phi1,  phi2,  phi3])

def _generate_four_event(startx, starty, theta0, phi0, zint,
                         theta1, phi1, theta2, phi2, theta3, phi3,
                         theta4, phi4):
    return _generate_event(startx, starty, theta0, phi0, zint,
                           4, [theta1, theta2, theta3, theta4],
                           [phi1,  phi2,  phi3,  phi4])


# ---------------------------------------------------------------------
# 2. Границы (bounds) для DE — отдельно на каждое N
#    • x0,y0  — [0,95]
#    • theta  — [0, π]
#    • phi    — [–π, π]
#    • zint   — [0, 44]  (при необходимости сузим потом)
# ---------------------------------------------------------------------
_bounds_xyz   = [(0, 95), (0, 95)]     # x0, y0
_bounds_theta_phi = [(0, np.pi), (-np.pi, np.pi)]  # одна пара (θ,φ)
_bounds_zint  = [(0, 44)]

bounds0 = _bounds_xyz + _bounds_theta_phi + _bounds_zint
bounds1 = _bounds_xyz + _bounds_theta_phi + _bounds_zint + \
          _bounds_theta_phi
bounds2 = _bounds_xyz + _bounds_theta_phi + _bounds_zint + \
          _bounds_theta_phi*2
bounds3 = _bounds_xyz + _bounds_theta_phi + _bounds_zint + \
          _bounds_theta_phi*3
bounds4 = _bounds_xyz + _bounds_theta_phi + _bounds_zint + \
          _bounds_theta_phi*4


# ---------------------------------------------------------------------
# 3. Быстрый Chamfer-loss (без POT) — подойдёт для сравнения оптимизаторов
# ---------------------------------------------------------------------
from scipy.spatial import cKDTree
def chamfer_loss(mask1: np.ndarray, mask2: np.ndarray) -> float:
    p1 = np.argwhere(mask1)
    p2 = np.argwhere(mask2)
    if len(p1) == 0 or len(p2) == 0:
        return np.inf
    d12 = cKDTree(p1).query(p2, k=1)[0].mean()
    d21 = cKDTree(p2).query(p1, k=1)[0].mean()
    return d12 + d21


# ---------------------------------------------------------------------
# 4. Универсальный «objective factory»
# ---------------------------------------------------------------------
def make_objective(gen_fun, target_mask):
    """gen_fun(**params) → mask; возвращает функцию f(params)->loss"""
    def f(params):
        p = list(params)
        # округляем координаты, где нужно
        p[0] = int(round(p[0]))   # x0
        p[1] = int(round(p[1]))   # y0
        p[4] = int(round(p[4]))   # zint
        try:
            gen_mask = gen_fun(*p) > 0
            return chamfer_loss(gen_mask, target_mask)
        except:
            return 1e6
    return f


# ---------------------------------------------------------------------
# 5. ПРИМЕР: восстанавливаем ту же target_mask разными моделями и
#            печатаем loss (чем меньше, тем лучше).
# ---------------------------------------------------------------------
if __name__ == "__main__":
    # --- тестовая цель (4-частичная) ---
    target_mask = (_generate_four_event(
        45, 45, np.pi/6, 0, 18,
        np.pi/6, 0,
        np.pi/6, np.pi/3,
        np.pi/8, -np.pi/4,
        np.pi/4, np.pi/6
    ) > 0)

    from scipy.optimize import differential_evolution

    for name, gen_fun, bounds in [
        ("N=0", _generate_zero_event,  bounds0),
        ("N=1", _generate_one_event,   bounds1),
        ("N=2", _generate_two_event,   bounds2),
        ("N=3", _generate_three_event, bounds3),
        ("N=4", _generate_four_event,  bounds4),
    ]:
        obj = make_objective(gen_fun, target_mask)
        res = differential_evolution(obj, bounds, popsize=5, maxiter=20, disp=False)
        print(f"{name:4s}  |  loss = {res.fun:.3f}")


N=0   |  loss = 11.520
N=1   |  loss = 8.349
N=2   |  loss = 7.733
N=3   |  loss = 7.681
N=4   |  loss = 6.827


In [31]:
from scipy.optimize import differential_evolution
import matplotlib.pyplot as plt

# Повторная оптимизация для N=1
obj = make_objective(_generate_one_event, target_mask)
result = differential_evolution(
    obj, bounds1,
    seed=42, maxiter=100, popsize=15, disp=True
)

# Полученные параметры
print("Recovered params:", result.x)
print("Loss:", result.fun)

# Полученная маска
recovered_mask = _generate_one_event(*result.x) > 0

# Визуализация по слоям z
for z in [10, 20, 30]:
    fig, ax = plt.subplots(1, 2, figsize=(8, 4))
    ax[0].imshow(target_mask[:, :, z], cmap='gray')
    ax[0].set_title(f'Target z={z}')
    ax[1].imshow(recovered_mask[:, :, z], cmap='gray')
    ax[1].set_title(f'Recovered z={z}')
    plt.show()


differential_evolution step 1: f(x)= 14.33061416210879
differential_evolution step 2: f(x)= 13.856609358190235
differential_evolution step 3: f(x)= 13.746094978667108
differential_evolution step 4: f(x)= 11.605139227906953
differential_evolution step 5: f(x)= 11.605139227906953
differential_evolution step 6: f(x)= 11.212898498434651
differential_evolution step 7: f(x)= 10.184282530717864
differential_evolution step 8: f(x)= 10.184282530717864
differential_evolution step 9: f(x)= 9.320166601949309
differential_evolution step 10: f(x)= 9.320166601949309
differential_evolution step 11: f(x)= 9.320166601949309
differential_evolution step 12: f(x)= 9.320166601949309
differential_evolution step 13: f(x)= 8.996207525502426
differential_evolution step 14: f(x)= 8.996207525502426
differential_evolution step 15: f(x)= 8.996207525502426
differential_evolution step 16: f(x)= 8.996207525502426
differential_evolution step 17: f(x)= 8.996207525502426
differential_evolution step 18: f(x)= 8.9962075255

In [83]:
import numpy as np
import pandas as pd

# 1. загрузка данных
calo = np.load("calorimeter_response.npy", allow_pickle=True).item()

# 2. статистика по событиям
ids, counts = np.unique(calo["event_ID"], return_counts=True)
df_stats = pd.DataFrame({"event_ID": ids, "hit_count": counts})

# 3. фильтрация звёздных событий (50–200 срабатываний)
star_df = df_stats[(df_stats.hit_count >= 50) & (df_stats.hit_count <= 200)]

# Вывод результата (альтернатива ace_tools)
print("События 50–200 hit:")
print(star_df)

События 50–200 hit:
      event_ID  hit_count
0          0.0         75
1          1.0        119
11        11.0         66
12        12.0        103
19        19.0        151
...        ...        ...
9884    9985.0        119
9891    9992.0        180
9892    9993.0        105
9894    9995.0        130
9898    9999.0         51

[3324 rows x 2 columns]


In [88]:
# ------------------------------------------------------------
# 0. Импорт + загрузка события  (EVENT_ID как раньше)
# ------------------------------------------------------------
import numpy as np, pandas as pd, matplotlib.pyplot as plt, ot, time
from scipy.optimize import differential_evolution, basinhopping
from scipy.spatial.distance import directed_hausdorff

payload = np.load("calorimeter_response.npy", allow_pickle=True).item()
hits = pd.DataFrame({c: payload[c] for c in
                     ["event_ID","layer","index_along_x","index_along_y","energy_release"]})

EVENT_ID = 1.0
evt = hits[hits.event_ID == EVENT_ID]

# 1) списки координат и энергий
coords_T, weight_T = [], []
for _,r in evt.iterrows():
    x,y,z = map(int,(r.index_along_x, r.index_along_y, r.layer))
    if 0<=x<96 and 0<=y<96 and 0<=z<44:
        coords_T.append((x,y,z))
        weight_T.append(r.energy_release)
coords_T = np.array(coords_T, dtype=float)
weight_T = np.array(weight_T, dtype=float)
weight_T /= weight_T.sum()         # нормируем веса

# ------------------------------------------------------------
# 1. energy-Wasserstein + базовые метрики
# ------------------------------------------------------------
def wemd(model_mask):
    P = np.argwhere(model_mask)        # точки модели
    if len(P)==0 or len(coords_T)==0: return 1e6
    a = weight_T
    b = np.ones(len(P)) / len(P)
    M = ot.dist(coords_T, P)           # евклид матрица расстояний
    return ot.emd2(a, b, M)            # EMD²

def haus(m):
    if len(m)==0: return np.inf
    d = directed_hausdorff(coords_T, m)[0]
    d2 = directed_hausdorff(m, coords_T)[0]
    return max(d,d2)

def iou(model_mask):
    # построим булев target-mask лишь для IoU/Dice
    target_bool = np.zeros((96,96,44), bool)
    for x,y,z in coords_T.astype(int): target_bool[x,y,z]=1
    inter = np.logical_and(target_bool, model_mask).sum()
    union = np.logical_or(target_bool, model_mask).sum()
    return inter/union if union else 0.

def dice(model_mask):
    target_bool = np.zeros((96,96,44), bool)
    for x,y,z in coords_T.astype(int): target_bool[x,y,z]=1
    inter = np.logical_and(target_bool, model_mask).sum()
    return 2*inter / (target_bool.sum()+model_mask.sum()+1e-8)

# ------------------------------------------------------------
# 2. генераторы + bounds  (как в предыдущем коде)
# ------------------------------------------------------------
# ...  (<- сюда копируем словари GEN и  BOUNDS  из прошлой ячейки) ...

def make_obj_energy(N):
    def f(p):
        p=list(p); p[0]=int(round(p[0])); p[1]=int(round(p[1])); p[4]=int(round(p[4]))
        try: mask = GEN[N](*p) > 0
        except: return 1e6
        return wemd(mask)
    return f

# ------------------------------------------------------------
# 3. оптимизируем N = 2..4  (энергетический loss)
# ------------------------------------------------------------
best = {}
for N in (2,3,4):
    print(f"\n=== ENERGY-WASSER, N={N}")
    objE = make_obj_energy(N)
    de  = differential_evolution(objE, BOUNDS[N], popsize=25, maxiter=200, disp=False, seed=0)
    bh  = basinhopping(objE, de.x,
                       minimizer_kwargs=dict(method='L-BFGS-B', bounds=BOUNDS[N]),
                       niter=30, seed=0)
    best[N] = dict(params=bh.x, loss=bh.fun)

bestN = min(best, key=lambda n: best[n]["loss"])
maskR = GEN[bestN](*best[bestN]["params"]) > 0

print(f"\n>> BEST by energy-WEMD:  N={bestN},  loss={best[bestN]['loss']:.2f}")
print("IoU  :", iou(maskR))
print("Dice :", dice(maskR))
print("Haus :", haus(maskR))

# ------------------------------------------------------------
# 4. Визуализация
# ------------------------------------------------------------
def proj(m, ttl):
    f,ax=plt.subplots(1,3,figsize=(10,3))
    ax[0].imshow(m.max(2)); ax[0].set_title('XY')
    ax[1].imshow(m.max(1)); ax[1].set_title('XZ')
    ax[2].imshow(m.max(0)); ax[2].set_title('YZ')
    f.suptitle(ttl); plt.tight_layout(); plt.show()

proj_bool = np.zeros((96,96,44), bool)
for x,y,z in coords_T.astype(int): proj_bool[x,y,z]=1

proj(proj_bool, "TARGET  (bool)")
proj(maskR,     f"RECOVERED energyW (N={bestN})")



=== ENERGY-WASSER, N=2


NameError: name 'BOUNDS' is not defined

In [85]:




def _generate_event(startx, starty, theta0, phi0, zint,
                    npart, theta_part, phi_part):
    """
    Возвращает бинарную маску формы (96, 96, 44).
    • primary-трек: идёт от z=0 до слоя int(zint) - 1 (вкл.)
    • каждое вторичное направление (θ_i, φ_i) пускаем до слоя 43.
    Все индексы проверяются, чтобы не выйти за границы.
    """
    mask = np.zeros((96, 96, 44), dtype=np.uint8)

    def propagate(x0, y0, theta, phi, z_stop):
        z, x, y = 0, x0, y0
        while (z < z_stop
               and 0 <= z < 44
               and 0 <= x < 96 and 0 <= y < 96):
            mask[int(x), int(y), int(z)] = 1
            # движение на следующий слой
            x += np.tan(theta) * np.cos(phi)
            y += np.tan(theta) * np.sin(phi)
            z += 1

    # ---------- primary ----------
    z_stop_primary = max(int(zint) - 1, 0)   # ключевая правка!
    propagate(startx, starty, theta0, phi0, z_stop_primary)

    # ---------- secondaries ----------
    for th, ph in zip(theta_part, phi_part):
        propagate(startx, starty, th, ph, 44)   # до конца объёма

    return mask  # (96, 96, 44)

# --------------------------------------------------------------------
# 2. ОБЁРТКА ДЛЯ МОДЕЛИ С 4 ВТОРИЧНЫМИ ЧАСТИЦАМИ
# --------------------------------------------------------------------
def _generate_four_event(startx, starty, theta0, phi0, zint,
                         theta1, phi1, theta2, phi2,
                         theta3, phi3, theta4, phi4):
    return _generate_event(
        startx, starty, theta0, phi0, zint,
        4,
        [theta1, theta2, theta3, theta4],
        [phi1,  phi2,  phi3,  phi4]
    )

# --------------------------------------------------------------------
# 3. АКТУАЛЬНЫЕ ГРАНИЦЫ (bounds) ДЛЯ Differential Evolution
#    13 параметров: (x0, y0, θ0, φ0, zint, θ1, φ1, …, θ4, φ4)
# --------------------------------------------------------------------
bounds_four = [
    (0, 95),            # startx
    (0, 95),            # starty
    (0, np.pi),         # theta0        – [0, π]   (можно и назад)
    (-np.pi, np.pi),    # phi0          – [-π, π]

    (0, 44),            # zint          – слой взаимодействия

    # ------- four secondaries -------
    (0, np.pi), (-np.pi, np.pi),   # θ1, φ1
    (0, np.pi), (-np.pi, np.pi),   # θ2, φ2
    (0, np.pi), (-np.pi, np.pi),   # θ3, φ3
    (0, np.pi), (-np.pi, np.pi),   # θ4, φ4
]


def _objective_four(params, target_mask):
    # --- круглые координаты ----------------------------
    params = list(params)                  # чтобы было изменяемо
    params[0] = int(round(params[0]))      # startx
    params[1] = int(round(params[1]))      # starty
    params[4] = int(round(params[4]))      # zint
    params = tuple(params)                 # обратно к tuple

    if len(params) != 13:
        return 1e6

    try:
        gen_mask = _generate_four_event(*params) > 0
        return wasserstein_distance(gen_mask, target_mask)
    except Exception as e:
        # отлавливаем — штраф
        # print("ошибка:", e)
        return 1e6



In [23]:
target_mask = (_generate_four_event(
    45, 45, np.pi/6, 0, 18,
    np.pi/6, 0,
    np.pi/6, np.pi/3,
    np.pi/8, -np.pi/4,
    np.pi/4, np.pi/6
) > 0)




result = differential_evolution(
    func=_objective_four,
    args=(target_mask,),
    bounds=bounds_four,      # <- ВАЖНО!
    strategy='best1bin',
    maxiter=100,             # можно 100-150, будет лучше садиться
    popsize=25,              # чуть больше популяция → лучше разведка
    mutation=(0.4, 1.0),
    recombination=0.7,
    disp=True,
    seed=42,
    polish=True              # L-BFGS-B в конце
)
print("параметры:", result.x)
print("отклонение:", result.fun)


differential_evolution step 1: f(x)= 8.896752570199379
differential_evolution step 2: f(x)= 8.896752570199379
differential_evolution step 3: f(x)= 8.395656871695227
differential_evolution step 4: f(x)= 8.081121022912544
differential_evolution step 5: f(x)= 6.636735021373165
differential_evolution step 6: f(x)= 6.636735021373165
differential_evolution step 7: f(x)= 5.305258971121186
differential_evolution step 8: f(x)= 5.305258971121186
differential_evolution step 9: f(x)= 5.305258971121186
differential_evolution step 10: f(x)= 5.305258971121186
differential_evolution step 11: f(x)= 5.305258971121186
differential_evolution step 12: f(x)= 5.305258971121186
differential_evolution step 13: f(x)= 5.305258971121186
differential_evolution step 14: f(x)= 5.305258971121186
differential_evolution step 15: f(x)= 5.305258971121186
differential_evolution step 16: f(x)= 5.305258971121186
differential_evolution step 17: f(x)= 4.915953835773851
differential_evolution step 18: f(x)= 4.915953835773851
d

In [87]:
import numpy as np
from scipy.spatial.distance import cdist, directed_hausdorff

# ---------------------------------------------------
# 1. Chamfer distance (двусторонняя)
# ---------------------------------------------------
def chamfer_distance(mask1: np.ndarray, mask2: np.ndarray):
    points1 = np.argwhere(mask1)
    points2 = np.argwhere(mask2)
    if len(points1) == 0 or len(points2) == 0:
        return np.inf

    dist12 = cdist(points1, points2)
    d1 = np.mean(np.min(dist12, axis=1))
    d2 = np.mean(np.min(dist12, axis=0))
    return d1 + d2

# ---------------------------------------------------
# 2. IoU и Dice
# ---------------------------------------------------
def iou(mask1: np.ndarray, mask2: np.ndarray):
    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    return intersection / union if union > 0 else 0.0

def dice(mask1: np.ndarray, mask2: np.ndarray):
    intersection = np.logical_and(mask1, mask2).sum()
    total = mask1.sum() + mask2.sum()
    return 2 * intersection / total if total > 0 else 0.0

# ---------------------------------------------------
# 3. Hausdorff distance (двусторонняя)
# ---------------------------------------------------
def hausdorff_distance(mask1: np.ndarray, mask2: np.ndarray):
    points1 = np.argwhere(mask1)
    points2 = np.argwhere(mask2)
    if len(points1) == 0 or len(points2) == 0:
        return np.inf

    h1 = directed_hausdorff(points1, points2)[0]
    h2 = directed_hausdorff(points2, points1)[0]
    return max(h1, h2)

# ---------------------------------------------------
# 4. Пример использования
# ---------------------------------------------------
# Предположим, есть две маски одинакового размера:
# (например, результат модели и "эталон")

# mask_true = ...   # ground truth (например, target_mask)
# mask_pred = ...   # результат модели (например, gen_mask)

def compare_masks(mask_true: np.ndarray, mask_pred: np.ndarray):
    print("Chamfer distance:", chamfer_distance(mask_true, mask_pred))
    print("Hausdorff distance:", hausdorff_distance(mask_true, mask_pred))
    print("IoU:", iou(mask_true, mask_pred))
    print("Dice:", dice(mask_true, mask_pred))



result = differential_evolution(
    func=_objective_four,
    args=(target_mask,),
    bounds=bounds_four,
    strategy='best1bin',
    maxiter=200,      # увеличили
    popsize=40,       # увеличили
    mutation=(0.4, 1.0),
    recombination=0.7,
    disp=True,
    seed=42,
    polish=True
)
params = list(result.x)
params[0] = int(round(params[0]))  # x
params[1] = int(round(params[1]))  # y
params[4] = int(round(params[4]))  # zint
params = tuple(params)

recovered_mask = _generate_four_event(*params) > 0

metrics = compare_masks(target_mask, recovered_mask)
metrics

differential_evolution step 1: f(x)= 10.778828311282474
differential_evolution step 2: f(x)= 7.1259406365874955
differential_evolution step 3: f(x)= 6.559465505549853
differential_evolution step 4: f(x)= 6.559465505549853
differential_evolution step 5: f(x)= 6.559465505549853
differential_evolution step 6: f(x)= 6.559465505549853
differential_evolution step 7: f(x)= 6.559465505549853
differential_evolution step 8: f(x)= 6.559465505549853
differential_evolution step 9: f(x)= 6.559465505549853
differential_evolution step 10: f(x)= 6.559465505549853
differential_evolution step 11: f(x)= 6.012513347828023
differential_evolution step 12: f(x)= 6.012513347828023
differential_evolution step 13: f(x)= 5.269594539821229
differential_evolution step 14: f(x)= 5.269594539821229
differential_evolution step 15: f(x)= 5.269594539821229
differential_evolution step 16: f(x)= 5.269594539821229
differential_evolution step 17: f(x)= 5.269594539821229
differential_evolution step 18: f(x)= 4.564150366181687

In [ ]:
import time
from scipy.optimize import basinhopping, shgo

# Объективная функция с явным target_mask
def make_objective(target_mask):
    def f(params):
        params = list(params)
        params[0] = int(round(params[0]))
        params[1] = int(round(params[1]))
        params[4] = int(round(params[4]))
        try:
            gen_mask = _generate_four_event(*params) > 0
            return wasserstein_distance(gen_mask, target_mask)
        except Exception as e:
            return 1e6
    return f

objective = make_objective(target_mask)

# ========== 1. differential_evolution ==========
start = time.time()
result_de = differential_evolution(
    func=objective,
    bounds=bounds_four,
    strategy='best1bin',
    maxiter=100,
    popsize=30,
    seed=0,
    polish=True
)
time_de = time.time() - start

print("=== differential_evolution ===")
print("loss =", result_de.fun)
print("time =", time_de)

# ========== 2. basinhopping ==========
start = time.time()
x0 = result_de.x  # стартуем с результата DE
result_bh = basinhopping(
    objective,
    x0=x0,
    niter=30,
    stepsize=1.0,
    minimizer_kwargs={'method': 'L-BFGS-B', 'bounds': bounds_four},
    seed=0
)
time_bh = time.time() - start

print("\n=== basinhopping ===")
print("loss =", result_bh.fun)
print("time =", time_bh)

# ========== 3. SHGO (Simplicial Homology Global Optimization) ==========
start = time.time()
result_shgo = shgo(
    func=objective,
    bounds=bounds_four,
    sampling_method='sobol'
)
time_shgo = time.time() - start

print("\n=== SHGO ===")
print("loss =", result_shgo.fun)
print("time =", time_shgo)


=== differential_evolution ===
loss = 0.04477692543562795
time = 169.00645804405212

=== basinhopping ===
loss = 0.04477692543562795
time = 5.614654302597046


In [ ]:
def _generate_event(startx, starty, theta0, phi0, zint,
                    npart, theta_part, phi_part):

    mask = np.zeros((96, 96, 44), dtype=np.uint8)

    def propagate(x0, y0, theta, phi, z_stop):
        z, x, y = 0, x0, y0
        while (z < z_stop
               and 0 <= z < 44
               and 0 <= x < 96 and 0 <= y < 96):
            mask[int(x), int(y), int(z)] = 1
           
            x += np.tan(theta) * np.cos(phi)
            y += np.tan(theta) * np.sin(phi)
            z += 1

    z_stop_primary = max(int(zint) - 1, 0)   
    propagate(startx, starty, theta0, phi0, z_stop_primary)

    for th, ph in zip(theta_part, phi_part):
        propagate(startx, starty, th, ph, 44)   

    return mask  


def _generate_four_event(startx, starty, theta0, phi0, zint,
                         theta1, phi1, theta2, phi2,
                         theta3, phi3, theta4, phi4):
    return _generate_event(
        startx, starty, theta0, phi0, zint,
        4,
        [theta1, theta2, theta3, theta4],
        [phi1,  phi2,  phi3,  phi4]
    )


bounds_four = [
    (0, 95),           
    (0, 95),            
    (0, np.pi),         
    (-np.pi, np.pi),    

    (0, 44),          
    (0, np.pi), (-np.pi, np.pi),   
    (0, np.pi), (-np.pi, np.pi),   
    (0, np.pi), (-np.pi, np.pi),  
    (0, np.pi), (-np.pi, np.pi),   
]


def _objective_four(params, target_mask):
  
    params = list(params)                 
    params[0] = int(round(params[0]))     
    params[1] = int(round(params[1]))     
    params[4] = int(round(params[4]))      
    params = tuple(params)                 

    if len(params) != 13:
        return 1e6

    try:
        gen_mask = _generate_four_event(*params) > 0
        return wasserstein_distance(gen_mask, target_mask)
    except Exception as e:
      
        return 1e6


target_mask = (_generate_four_event(
    45, 45, np.pi/6, 0, 18,
    np.pi/6, 0,
    np.pi/6, np.pi/3,
    np.pi/8, -np.pi/4,
    np.pi/4, np.pi/6
) > 0)




result = differential_evolution(
    func=_objective_four,
    args=(target_mask,),
    bounds=bounds_four,     
    strategy='best1bin',
    maxiter=100,            
    popsize=25,             
    mutation=(0.4, 1.0),
    recombination=0.7,
    disp=True,
    seed=42,
    polish=True             
)
print("параметры:", result.x)
print("отклонение:", result.fun)


In [63]:
import numpy as np

# --------------------------------------------------------------------
# 1. ГЕНЕРАТОР 3-D МАСКИ СО СТАБИЛЬНЫМ zint
# --------------------------------------------------------------------
def _generate_event(startx, starty, theta0, phi0, zint,
                    npart, theta_part, phi_part):
    """
    Возвращает бинарную маску формы (96, 96, 44).
    • primary-трек: идёт от z=0 до слоя int(zint) - 1 (вкл.)
    • каждое вторичное направление (θ_i, φ_i) пускаем до слоя 43.
    Все индексы проверяются, чтобы не выйти за границы.
    """
    mask = np.zeros((96, 96, 44), dtype=np.uint8)

    def propagate(x0, y0, theta, phi, z_stop):
        z, x, y = 0, x0, y0
        while (z < z_stop
               and 0 <= z < 44
               and 0 <= x < 96 and 0 <= y < 96):
            mask[int(x), int(y), int(z)] = 1
            # движение на следующий слой
            x += np.tan(theta) * np.cos(phi)
            y += np.tan(theta) * np.sin(phi)
            z += 1

    # ---------- primary ----------
    z_stop_primary = max(int(zint) - 1, 0)   # ключевая правка!
    propagate(startx, starty, theta0, phi0, z_stop_primary)

    # ---------- secondaries ----------
    for th, ph in zip(theta_part, phi_part):
        propagate(startx, starty, th, ph, 44)   # до конца объёма

    return mask  # (96, 96, 44)

# --------------------------------------------------------------------
# 2. ОБЁРТКА ДЛЯ МОДЕЛИ С 4 ВТОРИЧНЫМИ ЧАСТИЦАМИ
# --------------------------------------------------------------------
def _generate_four_event(startx, starty, theta0, phi0, zint,
                         theta1, phi1, theta2, phi2,
                         theta3, phi3, theta4, phi4):
    return _generate_event(
        startx, starty, theta0, phi0, zint,
        4,
        [theta1, theta2, theta3, theta4],
        [phi1,  phi2,  phi3,  phi4]
    )

# --------------------------------------------------------------------
# 3. АКТУАЛЬНЫЕ ГРАНИЦЫ (bounds) ДЛЯ Differential Evolution
#    13 параметров: (x0, y0, θ0, φ0, zint, θ1, φ1, …, θ4, φ4)
# --------------------------------------------------------------------
bounds_four = [
    (0, 95),            # startx
    (0, 95),            # starty
    (0, np.pi),         # theta0        – [0, π]   (можно и назад)
    (-np.pi, np.pi),    # phi0          – [-π, π]

    (0, 44),            # zint          – слой взаимодействия

    # ------- four secondaries -------
    (0, np.pi), (-np.pi, np.pi),   # θ1, φ1
    (0, np.pi), (-np.pi, np.pi),   # θ2, φ2
    (0, np.pi), (-np.pi, np.pi),   # θ3, φ3
    (0, np.pi), (-np.pi, np.pi),   # θ4, φ4
]

# --------------------------------------------------------------------
# 4. ПРИМЕР БЫСТРОГО ТЕСТА
#    (убедитесь, что всё строится корректно)
# --------------------------------------------------------------------
if __name__ == "__main__":
    # Синтетический вызов
    mask_demo = _generate_four_event(
        48, 48,               # x0, y0
        np.pi/4, 0.0,         # theta0, phi0
        18.6,                 # zint  (дробное – работает!)
        np.pi/6,  0.3,        # θ1, φ1
        np.pi/5, -0.7,        # θ2, φ2
        np.pi/4,  1.1,        # θ3, φ3
        np.pi/3, -2.0         # θ4, φ4
    )

    print("Shape:", mask_demo.shape)             # (96, 96, 44)
    print("Primary max-z:", np.max(np.argwhere(mask_demo)[:, 2]))  # ≤ int(18.6)-1 = 17


Shape: (96, 96, 44)
Primary max-z: 43


In [28]:
result = opt.minimize(_objective_zero, x0=[1.0, 1.0, np.pi/4, np.pi/6], args=(x_proj(Z), y_proj(Z)),
                      bounds=[(0, 96), (0, 96), (0, np.pi/3), (-np.pi, np.pi)],
                      callback=lambda result: print(".", end=""),
                      method="Powell")
#                       method='COBYLA')
#                       method='Nelder-Mead')

....

In [29]:
compare_XY(Z, _generate_zero_event(*result.x))

In [30]:
result = opt.minimize(_objective_one, x0=[34.0, 52.0, 0, 0, 10, 0, 0], args=(Z.any,),
                      bounds=[(0, 96), (0, 96), (0, np.pi/3), (-np.pi, np.pi), (0, 21), (-np.pi / 2, np.pi / 2), (-np.pi, np.pi)],
                      callback=lambda result: print(".", end=""),
                      method='Nelder-Mead')

TypeError: _objective_one() missing 1 required positional argument: 'to_y'

In [31]:
result = opt.minimize(_objective_three, x0=[34.0, 45.0, 0, 0, 10, 0.0, 0.0, 0.1, 0.1, 0.2, 0.2], args=(X,),
                      bounds=[(0, 95), (0, 95), (0, np.pi/3), (-np.pi, np.pi), (0, 21), (-np.pi / 2, np.pi / 2), (-np.pi, np.pi), (-np.pi / 2, np.pi / 2), (-np.pi, np.pi), (-np.pi / 2, np.pi / 2), (-np.pi, np.pi)],
                      callback=lambda result: print(".", end=""),
                      method='Nelder-Mead', tol=0.000000001)

TypeError: _objective_three() missing 1 required positional argument: 'to_y'

In [50]:
result = opt.minimize(_objective_three, x0=[34.0, 45.0, 0, 0, 10, 0.0, 0.0, 0.2, 0.2, 0.5, 0.5], args=(x_proj(X), y_proj(X)),
                      bounds=[(0, 95), (0, 95), (0, np.pi/3), (-np.pi, np.pi), (0, 21), (-np.pi / 2, np.pi / 2), (-np.pi, np.pi), (-np.pi / 2, np.pi / 2), (-np.pi, np.pi), (-np.pi / 2, np.pi / 2), (-np.pi, np.pi)],
                      callback=lambda result: print(".", end=""),
                      method='Nelder-Mead',
                      options={
                         'maxiter': 10000,
                         'maxfev': 20000,
                         'xatol': 1e-6,
                         'fatol': 1e-6,
                         'disp': True,
                         'adaptive': True
                     })

[34. 34. 34. 34. 34. 34. 34. 34. 34. 34. 34.] [45. 45. 45. 45. 45. 45. 45. 45. 45. 45. 45.]
[35.7 35.7 35.7 35.7 35.7 35.7 35.7 35.7 35.7 35.7 35.7] [45. 45. 45. 45. 45. 45. 45. 45. 45. 45. 45.]
[34. 34. 34. 34. 34. 34. 34. 34. 34. 34. 34.] [47.25 47.25 47.25 47.25 47.25 47.25 47.25 47.25 47.25 47.25 47.25]
[34.      34.00025 34.0005  34.00075 34.001   34.00125 34.0015  34.00175
 34.002   34.00225 34.0025 ] [45. 45. 45. 45. 45. 45. 45. 45. 45. 45. 45.]
[34. 34. 34. 34. 34. 34. 34. 34. 34. 34. 34.] [45. 45. 45. 45. 45. 45. 45. 45. 45. 45. 45.]
[34. 34. 34. 34. 34. 34. 34. 34. 34. 34. 34. 34.] [45. 45. 45. 45. 45. 45. 45. 45. 45. 45. 45. 45.]
[34. 34. 34. 34. 34. 34. 34. 34. 34. 34. 34.] [45. 45. 45. 45. 45. 45. 45. 45. 45. 45. 45.]
[34. 34. 34. 34. 34. 34. 34. 34. 34. 34. 34.] [45. 45. 45. 45. 45. 45. 45. 45. 45. 45. 45.]
[34. 34. 34. 34. 34. 34. 34. 34. 34. 34. 34.] [45. 45. 45. 45. 45. 45. 45. 45. 45. 45. 45.]
[34. 34. 34. 34. 34. 34. 34. 34. 34. 34. 34.] [45. 45. 45. 45. 45. 45. 45. 

KeyboardInterrupt: 

In [455]:
result = opt.minimize(_objective_three, x0=[34.0, 45.0, 0, 0, 10, 0.0, 0.0, 0.2, 0.2, -0.5, -0.5], args=(x_proj(X), y_proj(X)),
                      bounds=[(0, 95), (0, 95), (0, np.pi/3), (-np.pi, np.pi), (0, 21), (-np.pi / 2, np.pi / 2), (-np.pi, np.pi), (-np.pi / 2, np.pi / 2), (-np.pi, np.pi), (-np.pi / 2, np.pi / 2), (-np.pi, np.pi)],
                      callback=lambda result: print(".", end=""),
                      method='Powell',
                      options={
                         'maxiter': 10000,
                         'maxfev': 20000,
                         'xtol': 1e-6,
                         'ftol': 1e-6,
                         'disp': True
                     })

.....Optimization terminated successfully.
         Current function value: 3.006727
         Iterations: 5
         Function evaluations: 1606


In [43]:
result = opt.minimize(_objective_three, x0=[34.0, 45.0, 0, 0, 10, 0.0, 0.0, 0.2, 0.2, 0.5, 0.5], args=(x_proj(X), y_proj(X)),
                      bounds=[(0, 95), (0, 95), (0, np.pi/3), (-np.pi, np.pi), (0, 21), (-np.pi / 2, np.pi / 2), (-np.pi, np.pi), (-np.pi / 2, np.pi / 2), (-np.pi, np.pi), (-np.pi / 2, np.pi / 2), (-np.pi, np.pi)],
                      callback=lambda result: print(".", end=""),
                      method='cobyla',
                      options={
                         'maxiter': 100000,
                         'rhobeg': 0.2,
                         'tol': 1e-6,
                         'disp': True
                     })
                      # method='Powell')

........................................................................................................................................................

In [24]:
# Этот метод работает практически идеально, но долга

X = test_3_events[70]

def diff_callback(xk, convergence):
    current_min = _objective_three(xk, x_proj(X), y_proj(X))
    print(r"f(x) = {0:.4f}".format(current_min))
    return current_min < 0.1  # Stop if condition met

result = opt.differential_evolution(_objective_three, args=(x_proj(X), y_proj(X)),
                      bounds=[(0, 95), (0, 95), (0, np.pi/3), (-np.pi, np.pi), (0, 21), (-np.pi / 2, np.pi / 2), (-np.pi, np.pi), (-np.pi / 2, np.pi / 2), (-np.pi, np.pi), (-np.pi / 2, np.pi / 2), (-np.pi, np.pi)],
                      callback=diff_callback)
                      # method='Powell')

f(x) = 11.4854
f(x) = 10.5510
f(x) = 5.9564
f(x) = 5.9564
f(x) = 5.9564
f(x) = 5.9564
f(x) = 5.9564
f(x) = 5.8750
f(x) = 5.8750
f(x) = 5.8750
f(x) = 3.6905
f(x) = 3.6905
f(x) = 3.6905
f(x) = 3.6905
f(x) = 3.6905
f(x) = 3.6905
f(x) = 3.6905
f(x) = 3.6178
f(x) = 3.6178
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.5238
f(x) = 2.3412
f(x) = 2.3412
f(x) = 2.2514
f(x) = 2.2514
f(x) = 2.2514
f(x) = 2.2514
f(x) = 2.2514
f(x) = 2.2514
f(x) = 2.2514
f(x) = 2.2514
f(x) = 2.2153
f(x) = 2.2153
f(x) = 2.2153
f(x) = 2.2153
f(x) = 2.2153
f(x) = 2.2153
f(x) = 2.2153
f(x) = 2.2153
f(x) = 2.2153
f(x) = 2.2153
f(x) = 2.2153
f(x)

In [25]:
result

 message: callback function requested stop early by returning True
 success: False
     fun: 0.09523809523809523
       x: [ 4.373e+01  5.038e+01  2.633e-01 -4.700e-02  8.923e+00
           -5.119e-01  1.069e-01  5.880e-01 -8.394e-01 -1.462e+00
           -1.084e+00]
     nit: 289
    nfev: 47862

In [65]:
result.x

array([ 3.43309093e+01,  4.48236107e+01,  7.93908951e-01,  1.10535214e+00,
        7.08243411e+00, -1.60562142e-02,  2.10472424e+00,  1.20879001e+00,
       -2.18913559e+00, -1.03513583e+00,  2.38165634e+00])

In [26]:
compare_XY(X, _generate_three_event(*result.x))

In [66]:
compare_proj(X, _generate_three_event(*result.x))